# 2 Player FPSB Auction with symmetric unfirorm valuation distributions

## Imports

In [1]:
import os
import sys
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)

In [2]:
import torch
import torch.nn as nn
import torch.nn.utils as ut
from torch.optim.optimizer import Optimizer, required

In [3]:
from bnelearn.strategy import NeuralNetStrategy, TruthfulStrategy
from bnelearn.bidder import Bidder
from bnelearn.mechanism import FirstPriceSealedBidAuction, VickreyAuction
from bnelearn.optimizer import ES
from bnelearn.environment import AuctionEnvironment

In [4]:
from tensorboardX import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

In [5]:
torch.cuda.is_available()

True

## Settings

In [6]:
run_dir = 'fpsp'
run_name = 'test-experiment1'
logdir = os.path.join(root_path, 'notebooks', run_dir , run_name)

In [7]:
## Experiment setup
n_players = 2
n_items = 1
# valuation distribution
u_lo =0
u_hi =10

## Environment settings
#training batch size
batch_size = 2**10
input_length = 1

# model architecture
size_hidden_layer = 10

# optimization params
epoch = 10000
learning_rate = 2e-2
lr_decay = True
lr_decay_every = 2000
lr_decay_factor = 0.8

sigma = .1 #ES noise parameter
n_perturbations = 64

# plot and log training options
plot_epoch = 50
plot_points = min(150, batch_size)
sample_points = torch.from_numpy(np.linspace(u_lo, u_hi, u_hi+1)).float().view(-1, n_items).cuda()

# tensorboard writer settings

## Setting up the Environment

In [8]:
# for evaluation
def optimal_bid(valuation):
    return valuation / 2

In [9]:
# Wrapper transforming a strategy to bidder, used by the optimizer
def strat_to_bidder(strategy, batch_size):
    return Bidder.uniform(u_lo,u_hi, strategy, batch_size = batch_size, n_players=2)

In [10]:
model = NeuralNetStrategy(input_length, size_hidden_layer = size_hidden_layer, requires_grad=False).cuda()

In [11]:
mechanism = FirstPriceSealedBidAuction(cuda = True)
env = AuctionEnvironment(mechanism,
                  agents = [], #dynamically built
                  max_env_size = 1, #
                  batch_size = batch_size,
                  n_players =2,
                  strategy_to_bidder_closure = strat_to_bidder
                 )
optimizer = ES(model=model, environment = env,
               lr = learning_rate, sigma=sigma, n_perturbations=n_perturbations)

In [12]:
def log_hyperparams(writer):
    writer.add_scalar('hyperparams/batch_size', batch_size)
    writer.add_scalar('hyperparams/size_hidden_layer', size_hidden_layer)
    writer.add_scalar('hyperparams/learning_rate', learning_rate)
    writer.add_scalar('hyperparams/sigma', sigma)
    writer.add_scalar('hyperparams/n_perturbations', n_perturbations)
    
    

## Training

In [13]:
torch.cuda.empty_cache()
writer = SummaryWriter(logdir)
log_hyperparams(writer)

for e in range(epoch+1):    
    
    # lr decay?
    if lr_decay and e % lr_decay_every == 0 and e > 0:
        learning_rate = learning_rate * lr_decay_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
        writer.add_scalar('hyperparams/learning_rate', learning_rate, e)
        
    # always: do optimizer step
    utility = -optimizer.step()
    writer.add_scalar('eval/utility', utility, e) 
    
    # plot + eval
    if e % plot_epoch == 0:
        # plot current function output
        bidder = strat_to_bidder(model, batch_size)
        bidder.draw_valuations_()
        v = bidder.valuations
        b = bidder.get_action()
        share = (b/v).mean()
        writer.add_scalar('eval/utility', utility, e)
        writer.add_scalar('eval/share', share, e)       
            
        print("Epoch {}: \tavg bid: {:2f}, \tutility: {:2f}".format(e, share, utility))                
        
        # subsample points and plot
        v = v.detach().cpu().numpy()[:plot_points]
        b= b.detach().cpu().numpy()[:plot_points]
        optimal = optimal_bid(v)        
        fig = plt.figure()
        plt.plot(v,b, 'o', v, optimal, 'r-')        
        writer.add_figure('eval/bid_function', fig, e)        
        plt.show()
        
        # first step: write model
        if e==0:
            writer.add_graph(model, bidder.valuations)        
    
   
        
torch.cuda.empty_cache()
writer.close()

C:\Users\stefan\Anaconda3\envs\bnelearn\lib\site-packages\torch\nn\functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 0: 	avg bid: 0.000000, 	utility: 0.000000
Epoch 50: 	avg bid: 0.548977, 	utility: 3.053226
Epoch 100: 	avg bid: 1.063215, 	utility: 1.800467
Epoch 150: 	avg bid: 1.524006, 	utility: 1.335690
Epoch 200: 	avg bid: 3.608670, 	utility: 0.790082
Epoch 250: 	avg bid: 1.608251, 	utility: 0.497652
Epoch 300: 	avg bid: 1.767040, 	utility: 0.389828
Epoch 350: 	avg bid: 1.582550, 	utility: 0.315281
Epoch 400: 	avg bid: 1.746561, 	utility: 0.356785
Epoch 450: 	avg bid: 1.534306, 	utility: 0.223930
Epoch 500: 	avg bid: 2.537699, 	utility: 0.320368
Epoch 550: 	avg bid: 1.548642, 	utility: 0.361472
Epoch 600: 	avg bid: 1.646958, 	utility: 0.428572
Epoch 650: 	avg bid: 1.480407, 	utility: 0.535519
Epoch 700: 	avg bid: 1.533018, 	utility: 0.729115
Epoch 750: 	avg bid: 1.294332, 	utility: 0.570339
Epoch 800: 	avg bid: 4.655233, 	utility: 0.732849
Epoch 850: 	avg bid: 1.349325, 	utility: 0.915714
Epoch 900: 	avg bid: 1.766014, 	utility: 0.914930
Epoch 950: 	avg bid: 1.451422, 	utility: 0.916101
Epo

In [14]:
logdir

'C:\\Users\\stefan\\dev\\bnelearn\\notebooks\\fpsp\\test-experiment1'